In [ ]:
import pandas as pd
import folium
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import os
import requests, io

In [ ]:
CLIENT_ID = os.environ.get('FOURSQUARE_ID')
CLIENT_SECRET = os.environ.get('FOURSQUARE_SECRET')

VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [ ]:
london_codes_all = pd.read_csv('London postcodes.csv')

In [ ]:
london_codes_all.head(5)

In [ ]:
london_relevant_columns = london_codes_all[['District','Ward', 'Constituency', 'Postcode district', 'Postcode', 'Latitude', 'Longitude']]

In [ ]:
london_relevant_columns.head()

In [ ]:
london_relevant_columns.shape

In [ ]:
consituencies = london_relevant_columns.groupby('Constituency').agg('mean').reset_index()

In [ ]:
consituencies

In [ ]:
latitude = 51.5074
longitude = -0.1278

In [ ]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, consituency in zip(consituencies['Latitude'], consituencies['Longitude'], consituencies['Constituency']):
    label = '{}'.format(consituency)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [ ]:
districts = london_relevant_columns.groupby('Postcode district').agg('mean').reset_index()

In [ ]:
districts

In [ ]:
map_london_code = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, district in zip(districts['Latitude'], districts['Longitude'], districts['Postcode district']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london_code)  
    
map_london_code

### Use foursquare to get local venues

In [ ]:
CLIENT_ID = os.environ.get('FOURSQUARE_ID')
CLIENT_SECRET = os.environ.get('FOURSQUARE_SECRET')

VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
london_venues_raw = getNearbyVenues(names=districts['Postcode district'],
                                   latitudes=districts['Latitude'],
                                   longitudes=districts['Longitude']
                                  )

In [ ]:
london_venues = london_venues_raw.copy()
london_venues.shape

In [ ]:
london_venues.head()

In [ ]:
london_venues[london_venues['Neighborhood']=='BR2']

In [ ]:
london_venues['Venue Category'].value_counts()

In [ ]:
london_venues.Venue.value_counts()

In [ ]:
# london_venues['Venue Category'] = london_venues['Venue']

### one hot encoding

In [ ]:
london_one_hot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

In [ ]:
london_one_hot.shape

In [ ]:
london_one_hot['Postcode district'] = london_venues['Neighborhood']

In [ ]:
london_one_hot.shape

In [ ]:
# london_one_hot.to_csv('london_one_hot.csv')

In [ ]:
london_grouped = london_one_hot.groupby('Postcode district').mean().reset_index()
london_grouped.shape

In [ ]:
london_grouped[london_grouped['Postcode district']=='BR2'].to_csv('br2.csv')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 30

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode district']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode district'] = london_grouped['Postcode district']

for ind in np.arange(london_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)


In [ ]:
neighborhoods_venues_sorted

In [ ]:
# set number of clusters
kclusters = 30

london_grouped_clustering = london_grouped.drop('Postcode district', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
len(kmeans.labels_)

In [ ]:
kmeans.labels_

In [ ]:
labels = kmeans.labels_
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', labels)

In [ ]:
neighborhoods_venues_sorted.shape

In [ ]:
london_data = districts.copy()

london_merged = london_data.merge(neighborhoods_venues_sorted, on='Postcode district')

In [ ]:
london_merged.head()

### Map of london clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Postcode district'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
london_merged.shape[1]

In [ ]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

In [ ]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

In [ ]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

In [ ]:
london_merged.loc[london_merged['Cluster Labels'] == 4, london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]